In [ ]:
%use plotly
import kotlinx.coroutines.*

val data = runBlocking { analyzeZoneFiles("./zones") }

In [ ]:
val sortedZones = data.sortedBy { r -> r.second.necessarySignatures }
val zoneStrings = sortedZones.map { (zone, _) -> IDN.toUnicode(zone) }


Plotly.plot {
    bar {
        x.strings = zoneStrings
        y.numbers = sortedZones.map { (_, stat) -> stat.necessarySignatures }
        type = TraceType.bar
    }

    layout {
        width = 1700
        height = 950
        title {
            text = "Signatures per zone"
        }
        xaxis {
            title {
                text = "Zone"
            }
        }
        yaxis {
            title {
                text = "Signatures"
                type = AxisType.log
            }
        }

    }
}

In [ ]:
import java.net.IDN

Plotly.plot {
    bar {
        showlegend = false
        x.strings = zoneStrings
        y.numbers = sortedZones.map { (_, stat) -> stat.necessarySignatures * 0.424 / 60 }
    }

    scatter {
        showlegend = false
        x.strings = listOf(zones.first(), zones.last())
        y.numbers = listOf(30, 30)
        mode = ScatterMode.lines
        name = "30 minutes"
    }

    layout {
        width = 1500 //width = 1700
        height = 950
        title {
            text = "Time to sign zone"
        }
        xaxis {
            title {
                text = "Zone"
            }
            tickvals = ListValue(
                "com",
                "xyz",
                "site",
                "amsterdam",
                "zip",
                "website",
                "gov",
                "paris",
                "education",
                "link",
                "support",
                "ninja",
                "city",
                "brussels",
                "mov",
                "cool",
                "science",
                "eco",
                "pub",
                "new",
                "dance",
                "tel",
                "domains",
                "shopping",
                "press",
                "adudhabi",
                "taxi",
                "ryukyu",
                "onl",
                "africa",
                "berlin",
                "vegas",
                "irish",
                "kim",
                "zuerich",
                "hamburg",
                "porn",
                "adult",
                "rio",
                "miami",
                "boston",
                "istanbul",
                "sucks",
                "politie",
                "youtube",
                "forum",
                "music",
                "apple",
                "honda",
                "aaa",
                "amazon",
                "aws",
                "book",
                "free",
                "imdb",
                "now",
                "deal",
                "sky",
                "wow",
                "bbc",
                "dell",
                "hbo",
                "philips",
                "safety",
                "yandex",
                "playstation",
                "office",
                "alibaba",
                "cern",
                "data",
                "eat",
                "fly",
                "meme",
                "stockholm",
                "tui",
                "mit",
                "drive",
                "java",
                "abc",
                "azure",
                "hotmail",
                "microsoft",
                "ice",
                "next",
                "smart",
            ).list
        }
        yaxis {
            title {
                text = "Time"
            }
            type = AxisType.log
            tickmode = TickMode.array
            tickvals = ListValue(0.1, 1, 10, 30, 100, 1020, 10080, 99360).list
            ticktext =
                ListValue(
                    "6 seconds",
                    "1 minute",
                    "10 minutes",
                    "30 minutes",
                    "1,5 hour",
                    "17 hours",
                    "7 days",
                    "69 days"
                ).list
        }
    }
}